# WeatherPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from config import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# How nmany cities in this analysis?
cnt = 1500

In [2]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = f"{url}appid={api_key}&units={units}&q="

## Generate Cities List

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=cnt)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=cnt)
lat_lngs = zip(lats, lngs)

In [4]:
lats

array([ -1.89892323,  33.40762705, -64.13139964, ..., -30.61866678,
        -9.71387186,  58.87682635])

In [5]:
lngs

array([ 179.73507701, -148.3799296 ,  -98.10999734, ...,  171.57634403,
         74.45011554, -174.22800047])

In [6]:
lat_and_lng = list(zip(lats,lngs))
lat_and_lng

[(-1.8989232297725778, 179.7350770138719),
 (33.40762704882002, -148.37992959506076),
 (-64.13139964261255, -98.10999733582105),
 (-33.73419602038374, 160.78819436188525),
 (61.04263288403041, 62.89093409711842),
 (46.51392603956219, 108.39188606251236),
 (84.50531262243774, -58.557626637129786),
 (1.4864550253484623, -149.86902712922142),
 (-49.30541817900719, -97.09446633705828),
 (81.52972210947675, -7.187054590269554),
 (-43.15358601000461, -136.07944929668264),
 (-1.1033596156199792, 21.310626984354627),
 (-0.08559642588268446, -86.28837528311165),
 (54.74102253233835, -114.1980443331897),
 (5.4770626977470585, 117.88742469538903),
 (-45.23175004690442, 179.38180669523877),
 (-44.849347505024525, -101.56876094753218),
 (-70.19578166413885, 78.50413969465143),
 (67.58257658756165, -40.77685021167656),
 (86.61561374388762, -138.63584687101252),
 (-26.512145754695496, -5.19479247464281),
 (20.464753297335875, 32.756240470588835),
 (-66.66841358867717, -159.396658851946),
 (-21.551573

In [7]:
# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    print(city)
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        
print(cities)

rungata
kahului
punta arenas
port macquarie
pionerskiy
darhan
upernavik
atuona
castro
illoqqortoormiut
rikitea
boende
san cristobal
barrhead
sandakan
waipawa
castro
busselton
tasiilaq
tuktoyaktuk
jamestown
marawi
mataura
pouembout
butaritari
yellowknife
ilebo
klaksvik
eyl
kamenskoye
barentsburg
lavrentiya
port hedland
nizhneyansk
victoria
yellowknife
saleaula
hermanus
georgetown
cooma
lagoa
lethem
jamestown
dikson
vaini
tiksi
nabire
nemuro
vaini
mehamn
sotouboua
mar del plata
butaritari
bengkulu
kyaikkami
molchanovo
namatanai
albany
grand gaube
oranjemund
saint george
port elizabeth
necochea
ovsyanka
albany
bairiki
qaanaaq
mar del plata
ribeira grande
meulaboh
cam ranh
leningradskiy
mao
geraldton
rocha
micheweni
umzimvubu
leczna
kapaa
thinadhoo
moba
romitan
albany
tura
burnie
conselheiro pena
port alfred
pemangkat
aripuana
port alfred
avarua
mar del plata
punta arenas
cayenne
ushuaia
sentyabrskiy
ust-tsilma
oranjemund
atuona
bluff
avarua
hasan abdal
illoqqortoormiut
hithadhoo
puerto de

cape town
ushuaia
mansa
dongsheng
michurinsk
atuona
nizhneyansk
juneau
vrangel
pandan
airai
dovers
haapiti
tuatapere
sahrak
znamenskoye
fortuna
wellington
beringovskiy
lompoc
hobart
vaini
kaeo
floriano
east london
carnarvon
praia da vitoria
lagoa
cabatuan
odweyne
busselton
sisimiut
hermanus
faanui
sinnamary
mataura
santa rosa
illoqqortoormiut
santa
mandalgovi
rikitea
zhuanghe
bluff
sentyabrskiy
dickinson
nizhneyansk
kapaa
khasan
kapaa
norman wells
marsh harbour
busselton
georgetown
hamilton
faanui
ordu
victoria
carlsbad
alice springs
ribeira grande
kefalos
nikolskoye
longyearbyen
mataura
rikitea
punta arenas
carnarvon
saint-philippe
vaini
albany
new norfolk
mbala
hithadhoo
arraial do cabo
ushuaia
bluff
jamestown
puerto ayora
lazaro cardenas
samusu
bredasdorp
rikitea
ushuaia
dwarka
muros
punta arenas
cape town
barrow
cape town
ushuaia
ponta do sol
butaritari
illoqqortoormiut
thompson
vaitupu
hami
lafia
yellowknife
barentsburg
marcona
tautira
kaitangata
pangnirtung
ancud
puerto ayora
sol

In [8]:
# Print the city count to confirm sufficient count
len(cities)

615

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [9]:
# set up lists to hold reponse info
cities2 = []
lats2 = []
temps = []
humids = []
clouds = []
winds = []
hems = []
orig_lats = []

count = 0

# Loop through the list of cities and perform a request for data on each
for city in cities:
    response = requests.get(query_url + city).json()
#    print(response)
    
    # Try to grab the weather data if the city is found
    try:
        lat2 = (response['coord']['lat'])
        temp = (response['main']['temp'])
        humid = (response['main']['humidity'])
        cloud = (response['clouds']['all'])
        wind = (response['wind']['speed'])
        count = count +1
        print(f"{count}: {city}")
        hem = "Nothern" if (lat2 > 0) else "Southern"
        print(f"located in {hem} hemisphere")    
        print(f"latitude is: {lat2}")
        print(f"temperature: {temp}")
        print(f"humidity is: {humid}")
        print(f"cloudiness is: {cloud}")
        print(f"wind speed is: {wind}")

        lats2.append(response['coord']['lat'])
        temps.append(response['main']['temp'])
        humids.append(response['main']['humidity'])
        clouds.append(response['clouds']['all'])
        winds.append(response['wind']['speed'])
        hems.append(hem)
        cities2.append(city)
     
        print()
        
    # Handle exceptions for a character that is not available in the Star Wars API
    except:
        print(f"{city} NOT FOUND")
        print()
        pass


rungata NOT FOUND

1: kahului
located in Nothern hemisphere
latitude is: 20.89
temperature: 83.64
humidity is: 43
cloudiness is: 40
wind speed is: 21.92

2: punta arenas
located in Southern hemisphere
latitude is: -53.15
temperature: 39.09
humidity is: 80
cloudiness is: 75
wind speed is: 13.87

3: port macquarie
located in Southern hemisphere
latitude is: -31.43
temperature: 53.01
humidity is: 98
cloudiness is: 85
wind speed is: 0.69

4: pionerskiy
located in Nothern hemisphere
latitude is: 54.95
temperature: 51.8
humidity is: 93
cloudiness is: 20
wind speed is: 6.71

5: darhan
located in Nothern hemisphere
latitude is: 49.49
temperature: 60.24
humidity is: 82
cloudiness is: 68
wind speed is: 0.85

6: upernavik
located in Nothern hemisphere
latitude is: 72.79
temperature: 46.26
humidity is: 75
cloudiness is: 68
wind speed is: 10.74

7: atuona
located in Southern hemisphere
latitude is: -9.8
temperature: 79.34
humidity is: 75
cloudiness is: 2
wind speed is: 17.87

8: castro
located in S

61: geraldton
located in Southern hemisphere
latitude is: -28.77
temperature: 41
humidity is: 93
cloudiness is: 0
wind speed is: 5.82

62: rocha
located in Southern hemisphere
latitude is: -34.48
temperature: 66.72
humidity is: 92
cloudiness is: 100
wind speed is: 5.48

63: micheweni
located in Southern hemisphere
latitude is: -4.97
temperature: 75.07
humidity is: 69
cloudiness is: 3
wind speed is: 14.52

umzimvubu NOT FOUND

64: leczna
located in Nothern hemisphere
latitude is: 51.3
temperature: 57.11
humidity is: 87
cloudiness is: 12
wind speed is: 4.7

65: kapaa
located in Nothern hemisphere
latitude is: 22.08
temperature: 82.17
humidity is: 74
cloudiness is: 75
wind speed is: 10.29

66: thinadhoo
located in Nothern hemisphere
latitude is: 0.53
temperature: 82.72
humidity is: 70
cloudiness is: 97
wind speed is: 5.99

67: moba
located in Nothern hemisphere
latitude is: 33.23
temperature: 61.9
humidity is: 100
cloudiness is: 100
wind speed is: 1.83

68: romitan
located in Nothern hemi

121: dhidhdhoo
located in Nothern hemisphere
latitude is: 6.88
temperature: 81.54
humidity is: 79
cloudiness is: 100
wind speed is: 15.9

122: kuytun
located in Nothern hemisphere
latitude is: 54.34
temperature: 56.68
humidity is: 77
cloudiness is: 0
wind speed is: 5.88

123: tommot
located in Nothern hemisphere
latitude is: 58.96
temperature: 61.16
humidity is: 90
cloudiness is: 100
wind speed is: 3.96

124: palencia
located in Nothern hemisphere
latitude is: 42.42
temperature: 65.34
humidity is: 79
cloudiness is: 35
wind speed is: 5.99

125: savelugu
located in Nothern hemisphere
latitude is: 9.62
temperature: 78.44
humidity is: 78
cloudiness is: 95
wind speed is: 6.71

126: nikolskoye
located in Nothern hemisphere
latitude is: 59.7
temperature: 52.84
humidity is: 71
cloudiness is: 20
wind speed is: 4.47

muzquiz NOT FOUND

127: ixtlan del rio
located in Nothern hemisphere
latitude is: 21.03
temperature: 78.01
humidity is: 69
cloudiness is: 47
wind speed is: 8.08

vaitupu NOT FOUND



181: santa quiteria
located in Southern hemisphere
latitude is: -4.33
temperature: 78.28
humidity is: 59
cloudiness is: 82
wind speed is: 4.74

182: paamiut
located in Nothern hemisphere
latitude is: 61.99
temperature: 47.7
humidity is: 91
cloudiness is: 100
wind speed is: 6.4

183: fortuna
located in Nothern hemisphere
latitude is: 40.6
temperature: 62.19
humidity is: 87
cloudiness is: 1
wind speed is: 16.11

184: coquimbo
located in Southern hemisphere
latitude is: -29.95
temperature: 53.6
humidity is: 81
cloudiness is: 86
wind speed is: 6.93

185: catio
located in Nothern hemisphere
latitude is: 11.28
temperature: 77.99
humidity is: 89
cloudiness is: 100
wind speed is: 6.58

186: brae
located in Nothern hemisphere
latitude is: 60.4
temperature: 53.6
humidity is: 93
cloudiness is: 3
wind speed is: 5.82

187: bredasdorp
located in Southern hemisphere
latitude is: -34.53
temperature: 44.6
humidity is: 100
cloudiness is: 0
wind speed is: 5.82

188: hamilton
located in Nothern hemisphere

241: moose factory
located in Nothern hemisphere
latitude is: 51.26
temperature: 82.4
humidity is: 42
cloudiness is: 1
wind speed is: 4.7

242: salalah
located in Nothern hemisphere
latitude is: 17.02
temperature: 78.8
humidity is: 94
cloudiness is: 75
wind speed is: 8.05

243: broome
located in Nothern hemisphere
latitude is: 42.25
temperature: 72.39
humidity is: 83
cloudiness is: 75
wind speed is: 4.7

244: manono
located in Southern hemisphere
latitude is: -7.3
temperature: 69.21
humidity is: 35
cloudiness is: 0
wind speed is: 5.17

245: oktyabrskiy
located in Nothern hemisphere
latitude is: 54.48
temperature: 56.39
humidity is: 96
cloudiness is: 94
wind speed is: 4.52

246: san patricio
located in Nothern hemisphere
latitude is: 19.22
temperature: 82.4
humidity is: 74
cloudiness is: 75
wind speed is: 6.8

247: vetluzhskiy
located in Nothern hemisphere
latitude is: 58.39
temperature: 52.23
humidity is: 97
cloudiness is: 96
wind speed is: 4.9

248: pevek
located in Nothern hemisphere

302: kushima
located in Nothern hemisphere
latitude is: 31.46
temperature: 78.48
humidity is: 83
cloudiness is: 75
wind speed is: 14.99

303: sao joao da barra
located in Southern hemisphere
latitude is: -21.64
temperature: 71.01
humidity is: 81
cloudiness is: 0
wind speed is: 10.29

304: canutama
located in Southern hemisphere
latitude is: -6.53
temperature: 81.3
humidity is: 59
cloudiness is: 42
wind speed is: 0.6

305: paka
located in Nothern hemisphere
latitude is: 4.64
temperature: 77
humidity is: 94
cloudiness is: 100
wind speed is: 2.24

306: souillac
located in Southern hemisphere
latitude is: -20.52
temperature: 68
humidity is: 64
cloudiness is: 75
wind speed is: 19.46

307: banjar
located in Southern hemisphere
latitude is: -8.19
temperature: 77
humidity is: 100
cloudiness is: 20
wind speed is: 16.11

308: tambopata
located in Southern hemisphere
latitude is: -12.73
temperature: 79.3
humidity is: 72
cloudiness is: 53
wind speed is: 2.8

309: cangzhou
located in Nothern hemisp

361: srednekolymsk
located in Nothern hemisphere
latitude is: 67.45
temperature: 84
humidity is: 32
cloudiness is: 70
wind speed is: 6.93

362: anadyr
located in Nothern hemisphere
latitude is: 64.75
temperature: 57.09
humidity is: 76
cloudiness is: 20
wind speed is: 4.47

363: atar
located in Nothern hemisphere
latitude is: 20.52
temperature: 96.39
humidity is: 30
cloudiness is: 0
wind speed is: 9.64

bur gabo NOT FOUND

364: bjorkelangen
located in Nothern hemisphere
latitude is: 59.88
temperature: 51.35
humidity is: 88
cloudiness is: 7
wind speed is: 0.2

365: lazaro cardenas
located in Nothern hemisphere
latitude is: 17.96
temperature: 85.19
humidity is: 67
cloudiness is: 41
wind speed is: 7.07

366: madimba
located in Southern hemisphere
latitude is: -10.4
temperature: 72.19
humidity is: 78
cloudiness is: 15
wind speed is: 12.39

367: balintore
located in Nothern hemisphere
latitude is: 57.76
temperature: 57.63
humidity is: 82
cloudiness is: 56
wind speed is: 10.29

368: konakovo


421: kodinsk
located in Nothern hemisphere
latitude is: 58.69
temperature: 54.34
humidity is: 83
cloudiness is: 76
wind speed is: 2.93

grand river south east NOT FOUND

422: setermoen
located in Nothern hemisphere
latitude is: 68.86
temperature: 48.2
humidity is: 93
cloudiness is: 100
wind speed is: 5.82

tumannyy NOT FOUND

423: longyearbyen
located in Nothern hemisphere
latitude is: 78.22
temperature: 46.4
humidity is: 93
cloudiness is: 75
wind speed is: 10.29

424: angoram
located in Southern hemisphere
latitude is: -4.06
temperature: 79.72
humidity is: 85
cloudiness is: 54
wind speed is: 2.93

425: prince rupert
located in Nothern hemisphere
latitude is: 54.32
temperature: 53.6
humidity is: 100
cloudiness is: 90
wind speed is: 6.93

426: alingsas
located in Nothern hemisphere
latitude is: 57.93
temperature: 54.09
humidity is: 82
cloudiness is: 75
wind speed is: 6.93

427: concepcion
located in Southern hemisphere
latitude is: -36.83
temperature: 51.8
humidity is: 53
cloudiness is:

481: hobyo
located in Nothern hemisphere
latitude is: 5.35
temperature: 76.71
humidity is: 73
cloudiness is: 90
wind speed is: 26.4

482: kalemie
located in Southern hemisphere
latitude is: -5.95
temperature: 68.49
humidity is: 56
cloudiness is: 0
wind speed is: 3.94

483: nago
located in Nothern hemisphere
latitude is: 26.59
temperature: 84.13
humidity is: 83
cloudiness is: 1
wind speed is: 8.05

484: astaneh-ye ashrafiyeh
located in Nothern hemisphere
latitude is: 37.26
temperature: 73.4
humidity is: 100
cloudiness is: 10
wind speed is: 4.7

485: kindu
located in Southern hemisphere
latitude is: -2.95
temperature: 77.38
humidity is: 54
cloudiness is: 85
wind speed is: 1.32

486: morros
located in Southern hemisphere
latitude is: -9.45
temperature: 77.79
humidity is: 46
cloudiness is: 100
wind speed is: 6.73

487: ornskoldsvik
located in Nothern hemisphere
latitude is: 63.29
temperature: 51.8
humidity is: 93
cloudiness is: 90
wind speed is: 9.17

488: kamariotissa
located in Nothern h

541: constitucion
located in Southern hemisphere
latitude is: -35.33
temperature: 48.65
humidity is: 79
cloudiness is: 60
wind speed is: 3.36

542: veseloyarsk
located in Nothern hemisphere
latitude is: 51.28
temperature: 60.37
humidity is: 64
cloudiness is: 95
wind speed is: 4.43

543: besikduzu
located in Nothern hemisphere
latitude is: 41.05
temperature: 73.4
humidity is: 78
cloudiness is: 40
wind speed is: 2.24

544: teguise
located in Nothern hemisphere
latitude is: 29.06
temperature: 71.6
humidity is: 83
cloudiness is: 20
wind speed is: 18.34

545: calvinia
located in Southern hemisphere
latitude is: -31.47
temperature: 55.51
humidity is: 34
cloudiness is: 0
wind speed is: 9.4

546: bubaque
located in Nothern hemisphere
latitude is: 11.28
temperature: 80.69
humidity is: 83
cloudiness is: 100
wind speed is: 13.15

547: atambua
located in Southern hemisphere
latitude is: -9.11
temperature: 67.82
humidity is: 77
cloudiness is: 62
wind speed is: 6.17

548: alta floresta
located in So

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [15]:
# create a data frame from cities, lat, and temp
weather_dict = {
    "city": cities2,
    "hemisphere": hems,
    "lat": lats2,
    "temp": temps,
    "humidity": humids,
    "cloudiness": clouds,
    "wind_speed": winds
}
weather_data = pd.DataFrame(weather_dict)
weather_data

,city,hemisphere,lat,temp,humidity,cloudiness,wind_speed
0,kahului,Nothern,20.89,83.64,43,40,21.92
1,punta arenas,Southern,-53.15,39.09,80,75,13.87
2,port macquarie,Southern,-31.43,53.01,98,85,0.69
3,pionerskiy,Nothern,54.95,51.80,93,20,6.71
4,darhan,Nothern,49.49,60.24,82,68,0.85
...,...,...,...,...,...,...,...
560,bodden town,Nothern,19.28,78.55,94,75,3.36
561,wanlaweyn,Nothern,2.62,74.16,70,60,15.10
562,dong hoi,Nothern,17.48,81.00,72,34,2.01
563,port blair,Nothern,11.67,81.37,74,81,3.49


In [16]:
# Export file as a CSV, without the Pandas index, but with the header
weather_data.to_csv("Output/weather_data.csv", index=False, header=False)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [18]:
clean_city_data = weather_data.nlargest(5,'humidity')

In [26]:
high_humdity = weather_data['humidity'] > 100

In [27]:
high_humdity

0      False
1      False
2      False
3      False
4      False
       ...  
560    False
561    False
562    False
563    False
564    False
Name: humidity, Length: 565, dtype: bool

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression